# Semantic Similarity

In this notebook we follow a pipeline to extract relevant information from a data dump (generated by GPT).

The **Pipeline** is as follows:
* Read the Data
* Extract the Topics (We work with topics since it is easier to prototype with. Full Sentences can also be used.)
* Pre-Process the Data
* Generate Semantic Embeddings using a SentenceTransformer Model
* Perform K-Means Clustering

The last step, *Perform K-Means Clustering*, itself has a few parts:
* Optimize K for K-Means (this can be hard coded, or, as in the notebook, estimate from intertia)
* If preferred, apply Dimensionality Reduction using TSNE or PCA (this has proved to defeat the point of semantic embeddings to a large extent)
* Retrieve Cluster Assignments in the form of indexes
* Use the indexes to get the real clusters as a List of List of Strings (This is also where lexical cluster refinement has been built into)
* Print Clusters in a cohesive manner as well as the optimizations / datapoint filtering that has been done at multiple stages

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pylcs
import pandas as pd

In [ ]:
# Global variables for the pipeline

red1 = None
red2 = None

In [ ]:
# Read Dataset

with open('../data/I2_1_1000.txt','r') as txtfile:
    biz_ideas = [line.rstrip('\n') for line in txtfile]

In [ ]:
def topic(s1:str):
    
    # Returns the topic extracted from the entire output.
    
    return s1[:s1.find('.')]

In [ ]:
def preprocess(data:list, refer=None):
    
    # Applies three proprocessing steps: turns every item to lower case, selects only short strings, and eliminates duplicates
    
    data = [datum.lower() for datum in data]
    processed = {}
    
    if refer:
                
        for i in range(len(data)):
            
            if len(data[i]) < 25:
                processed[data[i]] = refer[i]
        
    else:
        data_processed = [datum for datum in data if len(datum) < 25]
        processed = dict(zip(data_processed, data_processed))
        
    global red1
    red1 = (len(data) - len(processed.keys()))/len(data) * 100
    
    return processed

In [ ]:
def overlap(s1:str, s2:str):
    
    # Returns the lexical overlap using longest common subsequence between two strings
    
    if len(s1) > len(s2):
        return pylcs.lcs(s1,s2)/float(len(s1))
    else:
        return pylcs.lcs(s1,s2)/float(len(s2))

In [ ]:
topics_unprocessed = []

for idea in biz_ideas:

    topics_unprocessed.append(topic(idea))
    
ideas = dict(zip())

processed = preprocess(topics_unprocessed,biz_ideas)

df = pd.DataFrame(zip(processed.values(), processed.keys()))
df.columns = ['Text','Topic']

# Create a list of topics from the corpus

topics = list(df['Topic'])

display(df)

In [ ]:
# Generate embeddings using a pre-trained SentenceTransformer model

model = SentenceTransformer('paraphrase-mpnet-base-v2')
topics_embeddings_unprocessed = model.encode(topics_unprocessed)
topics_embeddings = model.encode(topics)

print(f'Shape of topic embeddings before pre-processing: {topics_embeddings_unprocessed.shape}')
print(f'Shape of topic embeddings after pre-processing: {topics_embeddings.shape}')

In [ ]:
# Top K similar ideas (Pre-processed but unrefined)

topic_query = 'Beauty'
query_embedding = model.encode(topic_query)

top_k = 5

cos_scores = util.pytorch_cos_sim(query_embedding, topics_embeddings)[0]
top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

print("Sentence:", topic_query, "\n")
print(f'Top {top_k} most similar items in corpus:\n')

for idx in top_results[0:top_k]:
    print(topics[idx], "(Score: %.4f)" % (cos_scores[idx]))

In [ ]:
# Clustering : KMeans

def kmeans(num_clusters, data:list):
    
    from sklearn.cluster import KMeans
        
    clustering_model = KMeans(n_clusters = num_clusters)
    clustering_model.fit(data)
    cluster_assignment = clustering_model.labels_

    return cluster_assignment

In [ ]:
def optimize_k(data:list):
    
    # This function is used to automatically determine the optimal value of K for K-Means Clustering
    
    from sklearn.cluster import KMeans
    import math
    
    dists = []
    grads = []
    K = range(1,70)
    iner_drop = 0
    iner_last = 0
    
    for n in K:
        k_model = KMeans(n_clusters = n)
        k_model.fit(data)
        dists.append(k_model.inertia_)
        
    """ # This was an experiment to try stopping the optimization early to save time based on KMeans Inertia
        if n == 1:
            iner_last = k_model.inertia_
            grads.append(0)
        else:
            iner_drop = iner_last - k_model.inertia_
            grad = iner_drop/iner_last
            grads.append(grad)
            iner_last = k_model.inertia_
            
    for x, (y,z) in enumerate(zip(dists,grads),1):
        print(f'{x}. {y:.2f} {z:.4f}')
    """
    def calc_dist(x1,y1,a,b,c):
        return abs((a*x1 + b*y1 + c))/(math.sqrt(a**2 + b**2))
        
    a = dists[0] - dists[-1]
    b = K[-1] - K[0]
    c1 = K[0] * dists[-1]
    c2 = K[-1] * dists[0]
    c = c1 - c2
        
    dists_line = []

    for k in range(K[-1]):
        dists_line.append(calc_dist(K[k], dists[k], a, b, c))
                
    num_clusters = dists_line.index(max(dists_line))+1
        
    return num_clusters

In [ ]:
# Alternative Approach: DBScan

def dbscan(data:list):
    
    from sklearn.cluster import DBSCAN

    db_default = DBSCAN(eps = 0.0375, min_samples = 3).fit(data)
    cluster_assignment = db_default.labels_

    return cluster_assignment

In [ ]:
def reduce_dims(data, alg='tsne', num_components=2):
    
    # Used to reduce dimensions using TSNE or PCA algorithms
    
    topics_red = None
    
    if alg == 'tsne':
        
        from sklearn.manifold import TSNE

        topics_red = TSNE(n_components=num_components).fit_transform(data)
        
    elif alg == 'pca':
        
        from sklearn.decomposition import PCA
        topics_red = PCA(n_components=num_components,svd_solver='full').fit_transform(data)
            
    return topics_red

In [ ]:
def get_clusters(num_clusters, df):
    
    # Prepares and returns clusters as list of lists
    
    clusters = []
    
    for i in range(num_clusters):
        
        clust_sent = np.where(df['Cluster Assignment'] == i)
        clust_points = []
        
        for k in clust_sent[0]:
            
            clust_points.append(df['Topic'][k])
            
        clusters.append(clust_points)
    
    return clusters

In [ ]:
def refine_clusters(df, threshold=0.7):
    
    # Eliminates lexically similar items using Longest Common Subsequence
    
    refined_clusters = pd.DataFrame()
    reductions = []

    for i in range(num_clusters):

        df_cluster = df.where(df['Cluster Assignment'] == i+1)

        df_cluster.dropna(subset=['Text'], inplace=True)

        refined_cluster = pd.DataFrame()

        for j in range(df_cluster.shape[0]-1):
            flag = True

            for k in range(j+1,df_cluster.shape[0]):

                overlap_ = overlap(df_cluster.iloc[j]['Topic'],df_cluster.iloc[k]['Topic'])

                if overlap_ > 0.7:
                    #print('Hit')
                    flag = False
                    break

            if flag:
                    refined_cluster = refined_cluster.append(df_cluster.iloc[j])

        if df_cluster.shape[0]:
                reductions.append((df_cluster.shape[0] - refined_cluster.shape[0]) / df_cluster.shape[0])
                refined_clusters = refined_clusters.append(refined_cluster)

    global red2
    red2 = np.average(np.array(reductions))*100
    
    return refined_clusters

In [ ]:
def print_clusters(num_clusters, df, n=10):
    
    # Prints clusters in a cohesive manner and also displays how much redundant data has been eliminated

    for i in range(num_clusters):
        
        df_cluster = df.where(df['Cluster Assignment'] == i+1)
        df_cluster.dropna(subset=['Text'], inplace=True)
        display(df_cluster.head(n))
        
    global red1, red2
    print(f'\nData trimmed by {red1:.2f}% in preprocessing step, and by {red2:.2f}% in cluster refinement step.\n')

In [ ]:
%%time
# K-Means Pipeline

# Define number of clusters or auto estimate optimum using intertia
num_clusters = optimize_k(topics_embeddings)

# Reduce Dimentions using TSNE or PCA
topics_red = reduce_dims(topics_embeddings,alg='tsne',num_components=2)

# Apply K-Means Clustering
cluster_assignment = kmeans(num_clusters=num_clusters, data=topics_embeddings) # data=topics_embeddings, topics_red
df['Cluster Assignment'] = cluster_assignment

# Refine the clusters using LCS
clusters = refine_clusters(df)

# Print clusters cohesively
print_clusters(num_clusters, clusters, 5)

In [ ]:
# DBScan Pipeline

# Define number of clusters to show (note number of clusters is automatically determined)
num_clusters = 75

# Reduce Dimentions using TSNE
topics_red = reduce_dims(topics_embeddings,alg='tsne',num_components=3)

# Apply DBScan Clustering
cluster_assignment = dbscan(data=topics_embeddings) # data=topics_embeddings, topics_red

# Get the clusters
clusters = get_clusters(num_clusters, cluster_assignment, topics)

# Print clusters cohesively
print_clusters(clusters, 5)